In [6]:
import pandas as pd
import numpy as np
import string
import nltk
import re

In [7]:
def load_dataset():
    data = pd.read_csv('translated_dataset.csv')
    return data

tweet_df = load_dataset()
df = pd.DataFrame(tweet_df)

In [8]:
df

,Unnamed: 0,Unnamed: 0.1,timestamp,Tweet,translated_tweet
0,0,11190,2022-05-15 06:12:25+00:00,a-aku belum vaksin booster,I-I haven't had the booster vaccine yet
1,1,13777,2022-05-19 09:35:16+00:00,aa sudah vaksin booster belum boleh scan barco...,"aa already has the booster vaccine, you can't ..."
2,2,13749,2022-05-19 10:00:50+00:00,aamiin allahumma aamiin doa yang untuk klrga y...,"Amen, allahumma, amen, for the prayers for the..."
3,3,4648,2022-03-03 06:31:44+00:00,aamiin besok vaksin booster padahal,"Amen, tomorrow is a booster vaccine even though"
4,4,4518,2022-03-03 09:22:40+00:00,aamiin pulang vaksin langsung istirahat kak al...,"Amen, go home, take a break immediately, Sis, ..."
...,...,...,...,...,...
4055,4055,10424,2022-05-14 09:51:38+00:00,ayo yang vaksin booster ampul url,let's get vaccine booster ampoule url
4056,4056,4794,2022-03-03 03:00:35+00:00,ayo vaksin booster url,let's vaccine booster url
4057,4057,127,2021-11-25 06:57:09+00:00,iyai iyai yeay kookmin jikook short au bahasa ...,yes yes yes yes kookmin jikook short au people...
4058,4058,4632,2022-03-03 06:57:02+00:00,iya bebas pilih vaksin booster mana aja kalo n...,"Yes, you are free to choose any booster vaccin..."


In [9]:
df.drop(df.columns[[0,1,2,3]], axis=1, inplace=True)
df

,translated_tweet
0,I-I haven't had the booster vaccine yet
1,"aa already has the booster vaccine, you can't ..."
2,"Amen, allahumma, amen, for the prayers for the..."
3,"Amen, tomorrow is a booster vaccine even though"
4,"Amen, go home, take a break immediately, Sis, ..."
...,...
4055,let's get vaccine booster ampoule url
4056,let's vaccine booster url
4057,yes yes yes yes kookmin jikook short au people...
4058,"Yes, you are free to choose any booster vaccin..."


In [ ]:
def remove_pattern(tweet, pattern):
    r = re.findall(pattern, tweet)
    for i in r :
        tweet = re.sub(i, '', tweet)
    return tweet
df['remove'] = np.vectorize(remove_pattern)(df['translated_tweet'], "@[\w]*")

In [ ]:
from nltk.corpus import stopwords
stopwords= stopwords.words('indonesian', 'english')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
lemmatizer = nltk.stem.WordNetLemmatizer()

def clean_tweet(tweet):
    # remove angka
    tweet = re.sub('[0-9]+', '', tweet)

    # remove stock market
    tweet = re.sub(r'\$\w*', '', tweet)

    # remove RT
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hashtag
    tweet = re.sub(r'#', '', tweet)

    # remove hyperlink
    tweet = re.sub(r'https?:\/\/S+', '', tweet)

    # remove coma
    tweet = re.sub(r',', '', tweet)


    # tokenize
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_token = tokenizer.tokenize(tweet)

    tweet_clean = []
    for word in tweet_token:
        if (word not in stopwords and word not in string.punctuation):
            # stemming word
            stem_word = stemmer.stem(word) 
            tweet_clean.append(stem_word)

    return tweet_clean

In [ ]:
df['tweet_clean'] = df['remove'].apply(lambda x: clean_tweet(x))

In [ ]:
df

,translated_tweet,remove,tweet_clean
0,I-I haven't had the booster vaccine yet,I-I haven't had the booster vaccine yet,"[i-i, haven t, had, the, booster, vaccine, yet]"
1,"aa already has the booster vaccine, you can't ...","aa already has the booster vaccine, you can't ...","[aa, already, has, the, booster, vaccine, you,..."
2,"Amen, allahumma, amen, for the prayers for the...","Amen, allahumma, amen, for the prayers for the...","[amen, allahumma, amen, for, the, prayers, for..."
3,"Amen, tomorrow is a booster vaccine even though","Amen, tomorrow is a booster vaccine even though","[amen, tomorrow, is, a, booster, vaccine, even..."
4,"Amen, go home, take a break immediately, Sis, ...","Amen, go home, take a break immediately, Sis, ...","[amen, go, home, take, a, break, immediately, ..."
...,...,...,...
4055,let's get vaccine booster ampoule url,let's get vaccine booster ampoule url,"[let s, get, vaccine, booster, ampoule, url]"
4056,let's vaccine booster url,let's vaccine booster url,"[let s, vaccine, booster, url]"
4057,yes yes yes yes kookmin jikook short au people...,yes yes yes yes kookmin jikook short au people...,"[yes, yes, yes, yes, kookmin, jikook, short, a..."
4058,"Yes, you are free to choose any booster vaccin...","Yes, you are free to choose any booster vaccin...","[yes, you, are, free, to, choose, any, booster..."


In [ ]:
def remove_punc(text):
    text = ' '.join([char for char in text if char not in string.punctuation])
    return text

In [ ]:
df['tweet'] = df['tweet_clean'].apply(lambda x: remove_punc(x))

In [ ]:
df

,translated_tweet,remove,tweet_clean,tweet
0,I-I haven't had the booster vaccine yet,I-I haven't had the booster vaccine yet,"[i-i, haven t, had, the, booster, vaccine, yet]",i-i haven t had the booster vaccine yet
1,"aa already has the booster vaccine, you can't ...","aa already has the booster vaccine, you can't ...","[aa, already, has, the, booster, vaccine, you,...",aa already has the booster vaccine you can t s...
2,"Amen, allahumma, amen, for the prayers for the...","Amen, allahumma, amen, for the prayers for the...","[amen, allahumma, amen, for, the, prayers, for...",amen allahumma amen for the prayers for the fa...
3,"Amen, tomorrow is a booster vaccine even though","Amen, tomorrow is a booster vaccine even though","[amen, tomorrow, is, a, booster, vaccine, even...",amen tomorrow is a booster vaccine even though
4,"Amen, go home, take a break immediately, Sis, ...","Amen, go home, take a break immediately, Sis, ...","[amen, go, home, take, a, break, immediately, ...",amen go home take a break immediately sis when...
...,...,...,...,...
4055,let's get vaccine booster ampoule url,let's get vaccine booster ampoule url,"[let s, get, vaccine, booster, ampoule, url]",let s get vaccine booster ampoule url
4056,let's vaccine booster url,let's vaccine booster url,"[let s, vaccine, booster, url]",let s vaccine booster url
4057,yes yes yes yes kookmin jikook short au people...,yes yes yes yes kookmin jikook short au people...,"[yes, yes, yes, yes, kookmin, jikook, short, a...",yes yes yes yes kookmin jikook short au people...
4058,"Yes, you are free to choose any booster vaccin...","Yes, you are free to choose any booster vaccin...","[yes, you, are, free, to, choose, any, booster...",yes you are free to choose any booster vaccine...


In [ ]:
df.to_csv('label_dataset.csv', encoding='utf8', index=True)
df

,translated_tweet,remove,tweet_clean,tweet
0,I-I haven't had the booster vaccine yet,I-I haven't had the booster vaccine yet,"[i-i, haven t, had, the, booster, vaccine, yet]",i-i haven t had the booster vaccine yet
1,"aa already has the booster vaccine, you can't ...","aa already has the booster vaccine, you can't ...","[aa, already, has, the, booster, vaccine, you,...",aa already has the booster vaccine you can t s...
2,"Amen, allahumma, amen, for the prayers for the...","Amen, allahumma, amen, for the prayers for the...","[amen, allahumma, amen, for, the, prayers, for...",amen allahumma amen for the prayers for the fa...
3,"Amen, tomorrow is a booster vaccine even though","Amen, tomorrow is a booster vaccine even though","[amen, tomorrow, is, a, booster, vaccine, even...",amen tomorrow is a booster vaccine even though
4,"Amen, go home, take a break immediately, Sis, ...","Amen, go home, take a break immediately, Sis, ...","[amen, go, home, take, a, break, immediately, ...",amen go home take a break immediately sis when...
...,...,...,...,...
4055,let's get vaccine booster ampoule url,let's get vaccine booster ampoule url,"[let s, get, vaccine, booster, ampoule, url]",let s get vaccine booster ampoule url
4056,let's vaccine booster url,let's vaccine booster url,"[let s, vaccine, booster, url]",let s vaccine booster url
4057,yes yes yes yes kookmin jikook short au people...,yes yes yes yes kookmin jikook short au people...,"[yes, yes, yes, yes, kookmin, jikook, short, a...",yes yes yes yes kookmin jikook short au people...
4058,"Yes, you are free to choose any booster vaccin...","Yes, you are free to choose any booster vaccin...","[yes, you, are, free, to, choose, any, booster...",yes you are free to choose any booster vaccine...
